In [1]:
import plotly.express as px
import numpy as np

### Recopilation of indicators by page

In [2]:
codes = [
    "EDUNF_OFST_L1_UNDER1",
    "EDUNF_OFST_L1",
    "EDUNF_OFST_L2",
    "EDUNF_OFST_L3",
    "EDUNF_ROFST_L1",
    "EDUNF_ROFST_L2",
    "EDUNF_ROFST_L3",
    "EDUNF_STU_L1_TOT",
    "EDUNF_STU_L2_TOT",
    "EDUNF_STU_L3_TOT",
    "EDUNF_NER_L02",
    "EDUNF_NERA_L1_UNDER1",
    "EDUNF_NERA_L1",
    "EDUNF_NERA_L2",
    "EDUNF_NIR_L1_ENTRYAGE",
    "EDUNF_TRANRA_L2",
    "EDUNF_GER_L1",
    "EDUNF_GER_L2",
    "EDUNF_GER_L2_GEN",
    "EDUNF_GER_L2_VOC",
    "EDUNF_GER_L3",
    "EDUNF_GER_L3_GEN",
    "EDUNF_GER_L3_VOC",
    "EDU_SDG_SCH_L1",
    "EDU_SDG_SCH_L2",
    "EDU_SDG_SCH_L3",
    "EDU_CHLD_DISAB",
    "EDU_CHLD_DISAB_GENERAL",
    "EDU_CHLD_DISAB_SPECIAL",
    "WS_SCH_H-B",
    "WS_SCH_S-B",
    "WS_SCH_W-B",
    "EDUNF_CR_L1",
    "EDUNF_CR_L2",
    "EDUNF_CR_L3",
    "EDUNF_DR_L1",
    "EDUNF_DR_L2",
    "EDUNF_RPTR_L1",
    "EDUNF_RPTR_L2",
    "EDUNF_ESL_L1",
    "EDUNF_ADMIN_L1_GLAST_REA",
    "EDUNF_ADMIN_L1_GLAST_MAT",
    "EDUNF_ADMIN_L2_REA",
    "EDUNF_ADMIN_L2_MAT",
    "EDUNF_ADMIN_L1_G2OR3_REA",
    "EDUNF_ADMIN_L1_G2OR3_MAT",
    "EDU_PISA_MAT",
    "EDU_PISA_REA",
    "EDU_PISA_SCI",
    "ECD_CHLD_36-59M_LMPSL",
    "EDU_SDG_STU_L2_GLAST_MAT",
    "EDU_SDG_STU_L2_GLAST_REA",
    "EDU_SDG_STU_L1_GLAST_MAT",
    "EDU_SDG_STU_L1_G2OR3_MAT",
    "EDU_SDG_STU_L1_GLAST_REA",
    "EDU_SDG_STU_L1_G2OR3_REA",
    "EDUNF_LR_YOUTH",
    "EDUNF_LR_ADULT",
    "EDU_SDG_TRTP_L02",
    "EDU_SDG_TRTP_L1",
    "EDU_SDG_TRTP_L2",
    "EDU_SDG_TRTP_L3",
    "EDU_SDG_QUTP_L02",
    "EDU_SDG_QUTP_L1",
    "EDU_SDG_QUTP_L2",
    "EDU_SDG_QUTP_L3",
    "EDU_SDG_FREE_EDU_L02",
    "EDU_SDG_COMP_EDU_L02",
    "EDUNF_STU_L01_PUB",
    "EDUNF_STU_L02_PUB",
    "EDUNF_STU_L1_PUB",
    "EDUNF_STU_L2_PUB",
    "EDUNF_STU_L3_PUB",
    "EDUNF_STU_L01_PRV",
    "EDUNF_STU_L02_PRV",
    "EDUNF_STU_L1_PRV",
    "EDUNF_STU_L2_PRV",
    "EDUNF_STU_L3_PRV",
    "EDUNF_TEACH_L1",
    "EDUNF_TEACH_L2",
    "EDUNF_TEACH_L3",
    "EDU_FIN_EXP_PT_GDP",
    "EDU_FIN_EXP_PT_TOT",
    "EDU_FIN_EXP_L02",
    "EDU_FIN_EXP_L1",
    "EDU_FIN_EXP_L2",
    "EDU_FIN_EXP_L3",
    "EDU_FIN_EXP_L4",
    "EDU_FIN_EXP_L5T8",
    "EDUNF_PRP_L02",
    "EDUNF_PRP_L1",
    "EDUNF_PRP_L2",
    "EDUNF_PRP_L3",
    # "EDU_PISA_MAT2",
    # "EDU_PISA_REA2",
    # "EDU_PISA_SCI2",
    # "EDU_SDG_GER_L01",
    # "EDUNF_SAP_L02",
    "EDUNF_SAP_L1T3",
    # "EDUNF_SAP_L2",
    # "EDU_CHLD_DISAB_L02",
    # "EDU_CHLD_DISAB_L1",
    # "EDU_CHLD_DISAB_L2",
    # "EDU_CHLD_DISAB_L3",
    "PT_CHLD_1-14_PS-PSY-V_CGVR",
    "PT_CHLD_INRESIDENTIAL",
    "PT_CHLD_DISAB_PUBLIC",
    # "PT_CHLD_NO_PARENTAL_CARE_PUBLIC",
    # "PT_CHLD_NONPUBLIC",
    # "PT_CHLD_INRESIDENTIAL_RATE_A",
    "PT_CHLD_INRESIDENTIAL_RATE_B",
    # "PT_CHLD_NO_PARENTAL_CARE_RATE",
    "PT_CHLD_INCARE_FOSTER",
    # "PT_CHLD_INCARE_FOSTER_RATE",
    "PT_CHLD_CARED_BY_FOSTER",
    "PT_CHLD_DISAB_FOSTER",
    "PT_CHLD_CARED_BY_FOSTER_RATE",
    "PT_CHLD_CARED_GUARDIAN",
    "PT_CHLD_DISAB_CARED_GUARDIAN",
    "PT_CHLD_CARED_GUARDIAN_RATE",
    "PT_CHLD_ENTEREDFOSTER",
    "PT_CHLD_GUARDIAN",
    "PT_CHLD_LEFTRESCARE",
    # "PT_CHLD_LEFTFOSTER",
    # "PT_CHLD_GUARDIAN_LEFT",
    "PT_CHLD_ADOPTION",
    "PT_CHLD_ADOPTION_DISAB",
    # "PT_CHLD_ADOPTION_INTERCOUNTRY",
    # "PT_CHLD_ADOPTION_INTER_COUNTRY_DISAB",
    "PT_CHLD_ADOPTION_AVAILABLE",
    "PT_CHLD_ADOPTION_AVAILABLE_DISAB",
    # "PT_CHLD_ADOPTION_INTERCOUNTRY_RATE",
    "PT_CHLD_ADOPTION_RATE",
    "JJ_CHLD_CRIME",
    "JJ_CHLD_CRIMERT",
    "JJ_CHLD_DETENTION",
    "JJ_CHLD_CONVICTED",
    # "JJ_CHLD_CONVICTED_VIOLENT",
    # "JJ_CHLD_CONVICTED_PROPERTY",
    # "JJ_CHLD_CONVICTED_OTHER",
    "JJ_CHLD_SENTENCERT",
    # "JJ_CHLD_SENTENCE_PRISION",
    # "JJ_CHLD_SENTENCE_CORRECTIONAL",
    # "JJ_CHLD_SENTENCE_PROBATION",
    # "JJ_CHLD_SENTENCE_CUSTODIAL",
    # "JJ_CHLD_SENTENCE_FINANCIAL",
    # "JJ_CHLD_SENTENCE_SERVICE",
    # "JJ_CHLD_SENTENCE_LABOUR",
    # "JJ_CHLD_SENTENCE_OTHER",
    # "JJ_CHLD_PRISION",
    # "JJ_CHLD_PRETRIAL",
    # "JJ_CHLD_PRISION_ADJUDICATION",
    "JJ_CHLD_POLICE",
    "JJ_CHLD_OFFENCE",
    # "JJ_CHLD_CRIME_PERPETRATOR",
    # "JJ_CHLD_CRIME_PERSON",
    # "JJ_CHLD_CRIME_PROPERTY",
    # "JJ_CHLD_CRIME_OTHER",
    # "JJ_CHLD_ARRESTED",
    # "JJ_CHLD_VICTIM",
    # "JJ_CHLD_OFFENDER",
    "JJ_VC_PRS_UNSNT",
    "PT_ADLT_PS_NEC",
    "PT_CHLD_5-17_LBR_ECON",
    "PT_CHLD_5-17_LBR_ECON-HC",
    "PT_CHLD_LEFTRESCARE_ADOPTED",
    "PT_CHLD_LEFTRESCARE_DIED",
    "PT_CHLD_LEFTRESCARE_INDEPENDENT",
    "PT_CHLD_LEFTRESCARE_INFAMILY",
    "PT_CHLD_LEFTRESCARE_OTHER",
    "PT_CHLD_LEFTRESCARE_RETURNED",
    "PT_CHLD_LEFTRESCARE_TRANSFERED",
    "PT_F_15-19_MRD",
    "PT_F_15-49_W-BTNG",
    "PT_F_18-29_SX-V_AGE-18",
    "PT_F_20-24_MRD_U15",
    "PT_F_20-24_MRD_U18",
    "PT_F_GE15_PS-SX-EM_V_PTNR_12MNTH",
    "PT_F_GE15_SX_V_PTNR_12MNTH",
    "PT_M_15-19_MRD",
    "PT_M_15-49_W-BTNG",
    "PT_M_18-29_SX-V_AGE-18",
    "PT_M_20-24_MRD_U18",
    "PT_ST_13-15_BUL_30-DYS",
    "PT_VC_PRR_PHYV",
    "PT_VC_PRR_ROBB",
    "PT_VC_PRR_SEXV",
    "PT_VC_SNS_WALN",
    "PT_VC_VOV_PHYL",
    "PT_VC_VOV_ROBB",
    "PT_VC_VOV_SEXL",
]

#### Education config

In [3]:
edu_dict = {
    "PARTICIPATION": {
        "NAME": "Participation",
        "CARDS": [
            {
                "name": "Who are Out-of-School",
                "indicator": "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3",
                "suffix": "Primary to upper secondary aged Children and Adolescents",
            },
            {
                "name": "Who are Out-of-School",
                "indicator": "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3",
                "suffix": "Primary to upper secondary aged Girls",
                "sex": "Female",
            },
            {
                "name": "Who are Out-of-School",
                "indicator": "EDUNF_OFST_L1_UNDER1",
                "suffix": "Children one year younger than the official primary entry age",
            },
        ],
        "MAIN": {
            "name": "Out of School Children",
            "geo": "Geographic area",
            "options": dict(
                lat="latitude",
                lon="longitude",
                size="OBS_VALUE",
                text="Geographic area",
                color="OBS_VALUE",
                color_continuous_scale=px.colors.sequential.GnBu,
                size_max=40,
                zoom=2.5,
                animation_frame="TIME_PERIOD",
                height=750,
            ),
            "indicators": [
                "EDUNF_ROFST_L1",
                "EDUNF_ROFST_L2",
                "EDUNF_ROFST_L3",
                "EDUNF_OFST_L1",
                "EDUNF_OFST_L2",
                "EDUNF_OFST_L3",
            ],
            "default": "EDUNF_ROFST_L3",
        },
        "AREA_1": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD",
            ),
            "compare": "Sex",
            "indicators": [
                "EDUNF_ROFST_L1",
                "EDUNF_ROFST_L2",
                "EDUNF_ROFST_L3",
                "EDUNF_STU_L1_TOT",
                "EDUNF_STU_L2_TOT",
                "EDUNF_STU_L3_TOT",
            ],
            "default": "EDUNF_ROFST_L3",
        },
        "AREA_2": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "default_graph": "line",
            "indicators": [
                "EDUNF_NER_L02",
                "EDUNF_NERA_L1_UNDER1",
                "EDUNF_NERA_L1",
                "EDUNF_NERA_L2",
                "EDUNF_GER_L1",
                "EDUNF_GER_L2",
                "EDUNF_GER_L3",
                "EDUNF_NIR_L1_ENTRYAGE",
                "EDUNF_TRANRA_L2",
            ],
            # "default": "EDUNF_ROFST_L3",
        },
        "AREA_3": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD"
            ),
            "compare": "Sex",
            "indicators": [
                "EDU_SDG_SCH_L1",
                "EDU_SDG_SCH_L2",
                "EDU_SDG_SCH_L3",
                "WS_SCH_H-B",
                "WS_SCH_S-B",
                "WS_SCH_W-B",
                "EDU_CHLD_DISAB",
                "EDU_CHLD_DISAB_GENERAL",
                "EDU_CHLD_DISAB_SPECIAL",
            ],
            "default": "EDU_CHLD_DISAB",
        },
        "AREA_4": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "default_graph": "bar",
            "indicators": [
                "EDUNF_CR_L1",
                "EDUNF_CR_L2",
                "EDUNF_CR_L3",
                "EDUNF_DR_L1",
                "EDUNF_DR_L2",
                "EDUNF_GER_L2",
                "EDUNF_GER_L2_GEN",
                "EDUNF_GER_L2_VOC",
                "EDUNF_GER_L3",
                "EDUNF_GER_L3_GEN",
                "EDUNF_GER_L3_VOC",
            ],
            "default": "EDUNF_CR_L1",
        },
    },
    "QUALITY": {
        "NAME": "Learning Quality",
        "CARDS": [
            {
                "name": "(enrolled in the same grade for a second or further year) in primary and lower secondary education",
                "indicator": "EDUNF_RPTR_L1,EDUNF_RPTR_L2",
                "suffix": "Children and adolescent repeaters",
            },
            {
                "name": "from primary education",
                "indicator": "EDUNF_ESL_L1",
                "suffix": "Early school leavers",
            },
            {
                "name": "administering nationally representative learning assessment in both reading and math at the end of primary education",
                "indicator": "EDUNF_ADMIN_L1_GLAST_REA,EDUNF_ADMIN_L1_GLAST_MAT",
                "suffix": "Countries",
            },
            {
                "name": "participating in the latest round of PISA",
                "indicator": "EDU_PISA_MAT,EDU_PISA_REA,EDU_PISA_SCI",
                "suffix": "Countries",
                "absolute": True,
            },
        ],
        "MAIN": {
            "geo": "Geographic area",
            "options": dict(
                lat="latitude",
                lon="longitude",
                size="OBS_VALUE",
                text="Geographic area",
                color="OBS_VALUE",
                color_continuous_scale=px.colors.sequential.GnBu,
                size_max=40,
                zoom=2.5,
                animation_frame="TIME_PERIOD",
                height=750,
            ),
            "indicators": ["EDU_PISA_MAT", "EDU_PISA_REA", "EDU_PISA_SCI"],
            "default": "EDU_PISA_MAT",
        },
        "AREA_1": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD",
            ),
            "compare": "Sex",
            "indicators": [
                "ECD_CHLD_36-59M_LMPSL",
                "EDU_SDG_STU_L2_GLAST_MAT",
                "EDU_SDG_STU_L2_GLAST_REA",
                "EDU_SDG_STU_L1_GLAST_MAT",
                "EDU_SDG_STU_L1_G2OR3_MAT",
                "EDU_SDG_STU_L1_GLAST_REA",
                "EDU_SDG_STU_L1_G2OR3_REA",
                "EDUNF_LR_YOUTH",
                "EDUNF_LR_ADULT",
            ],
            "default": "EDU_SDG_STU_L2_GLAST_MAT",
        },
        "AREA_2": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "default_graph": "line",
            "indicators": [
                "ECD_CHLD_36-59M_LMPSL",
                "EDU_SDG_STU_L2_GLAST_MAT",
                "EDU_SDG_STU_L2_GLAST_REA",
                "EDU_SDG_STU_L1_GLAST_MAT",
                "EDU_SDG_STU_L1_G2OR3_MAT",
                "EDU_SDG_STU_L1_GLAST_REA",
                "EDU_SDG_STU_L1_G2OR3_REA",
                "EDUNF_LR_YOUTH",
                "EDUNF_LR_ADULT",
            ],
            # "default": "EDU_SDG_STU_L2_GLAST_MAT",
        },
        "AREA_3": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD"
            ),
            "compare": "Sex",
            "indicators": [
                "EDU_SDG_TRTP_L02",
                "EDU_SDG_TRTP_L1",
                "EDU_SDG_TRTP_L2",
                "EDU_SDG_TRTP_L3",
            ],
            "default": "EDU_SDG_TRTP_L2",
        },
        "AREA_4": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "default_graph": "bar",
            "indicators": [
                "EDU_SDG_QUTP_L02",
                "EDU_SDG_QUTP_L1",
                "EDU_SDG_QUTP_L2",
                "EDU_SDG_QUTP_L3",
            ],
            "default": "EDU_SDG_QUTP_L2",
        },
    },
    "GOVERNANCE": {
        "NAME": "Governance",
        "CARDS": [
            {
                "name": "Guaranteeing at least one year of free pre-primary education in their legal frameworks",
                "indicator": "EDU_SDG_FREE_EDU_L02",
                "suffix": "Countries",
            },
            {
                "name": "Enrolled in private institutions (primary, lower secondary and upper secondary education)",
                "indicator": "EDUNF_STU_L1_PRV,EDUNF_STU_L2_PRV,EDUNF_STU_L3_PRV",
                "suffix": "Children and Adolescents",
            },
            {
                "name": "Total in primary, lower secondary and upper secondary education",
                "indicator": "EDUNF_TEACH_L1,EDUNF_TEACH_L2,EDUNF_TEACH_L3",
                "suffix": "Classroom Teachers",
            },
        ],
        "MAIN": {
            "name": "Education Expenditures and Legal Frameworks",
            "geo": "Geographic area",
            "options": dict(
                lat="latitude",
                lon="longitude",
                size="OBS_VALUE",
                text="Geographic area",
                color="OBS_VALUE",
                color_continuous_scale=px.colors.sequential.GnBu,
                size_max=40,
                zoom=2.5,
                animation_frame="TIME_PERIOD",
                height=750,
            ),
            "indicators": [
                "EDU_FIN_EXP_PT_GDP",
                "EDU_FIN_EXP_PT_TOT",
                "EDU_SDG_FREE_EDU_L02",
                "EDU_SDG_COMP_EDU_L02",
            ],
            "default": "EDU_FIN_EXP_PT_GDP",
        },
        "AREA_1": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD",
            ),
            "compare": "Sex",
            "indicators": [
                "EDUNF_PRP_L02",
                "EDUNF_PRP_L1",
                "EDUNF_PRP_L2",
                "EDUNF_PRP_L3",
                "EDUNF_STU_L01_PUB",
                "EDUNF_STU_L02_PUB",
                "EDUNF_STU_L1_PUB",
                "EDUNF_STU_L2_PUB",
                "EDUNF_STU_L3_PUB",
                "EDUNF_STU_L01_PRV",
                "EDUNF_STU_L02_PRV",
                "EDUNF_STU_L1_PRV",
                "EDUNF_STU_L2_PRV",
                "EDUNF_STU_L3_PRV",
            ],
            "default": "EDUNF_PRP_L1",
        },
        "AREA_2": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "default_graph": "line",
            "indicators": [
                "EDUNF_PRP_L02",
                "EDUNF_PRP_L1",
                "EDUNF_PRP_L2",
                "EDUNF_PRP_L3",
                "EDUNF_STU_L01_PUB",
                "EDUNF_STU_L02_PUB",
                "EDUNF_STU_L1_PUB",
                "EDUNF_STU_L2_PUB",
                "EDUNF_STU_L3_PUB",
                "EDUNF_STU_L01_PRV",
                "EDUNF_STU_L02_PRV",
                "EDUNF_STU_L1_PRV",
                "EDUNF_STU_L2_PRV",
                "EDUNF_STU_L3_PRV",
            ],
            # "default": "EDUNF_PRP_L2",
        },
        "AREA_3": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD"
            ),
            "compare": "Sex",
            "indicators": [
                "EDU_FIN_EXP_L02",
                "EDU_FIN_EXP_L1",
                "EDU_FIN_EXP_L2",
                "EDU_FIN_EXP_L3",
                "EDU_FIN_EXP_L4",
                "EDU_FIN_EXP_L5T8",
            ],
            "default": "EDU_FIN_EXP_L2",
        },
        "AREA_4": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "default_graph": "bar",
            "indicators": [
                "EDUNF_ADMIN_L1_GLAST_REA",
                "EDUNF_ADMIN_L1_GLAST_MAT",
                "EDUNF_ADMIN_L2_REA",
                "EDUNF_ADMIN_L2_MAT",
                "EDUNF_ADMIN_L1_G2OR3_REA",
                "EDUNF_ADMIN_L1_G2OR3_MAT",
            ],
            "default": "EDUNF_ADMIN_L2_MAT",
        },
    },
}

#### CP config

In [4]:
cp_dict = {
    "VIOLENCE": {
        "NAME": "Violence against Children and Women",
        "CARDS": [
            # revise denominator population: children 1-14?
            {
                "name": "who experienced physical punishment or psychological aggression by caregivers",
                "indicator": "PT_CHLD_1-14_PS-PSY-V_CGVR",
                "denominator": "EDUNF_SAP_L1T3",
                "suffix": "Percent of Children",
            },
        ],
        "MAIN": {
            "name": "Violence and Harmful Practices",
            "geo": "Geographic area",
            "options": dict(
                lat="latitude",
                lon="longitude",
                size="OBS_VALUE",
                text="Geographic area",
                color="OBS_VALUE",
                color_continuous_scale=px.colors.sequential.GnBu,
                size_max=40,
                zoom=2.5,
                animation_frame="TIME_PERIOD",
                height=750,
            ),
            "indicators": [
                "PT_F_GE15_PS-SX-EM_V_PTNR_12MNTH",
                "PT_F_GE15_SX_V_PTNR_12MNTH",
                "PT_VC_VOV_PHYL",
                "PT_VC_VOV_SEXL",
                "PT_VC_VOV_ROBB",
                "PT_VC_SNS_WALN",
                "PT_CHLD_1-14_PS-PSY-V_CGVR",
                "PT_F_18-29_SX-V_AGE-18",
                "PT_M_18-29_SX-V_AGE-18",
                "PT_VC_PRR_PHYV",
                "PT_VC_PRR_SEXV",
                "PT_VC_PRR_ROBB",
                "PT_ADLT_PS_NEC",
                "PT_F_15-49_W-BTNG",
                "PT_M_15-49_W-BTNG",
                "PT_ST_13-15_BUL_30-DYS",
                "PT_F_15-19_MRD",
                "PT_M_15-19_MRD",
                "PT_F_20-24_MRD_U15",
                "PT_F_20-24_MRD_U18",
                "PT_M_20-24_MRD_U18",
                "PT_CHLD_5-17_LBR_ECON",
                "PT_CHLD_5-17_LBR_ECON-HC",
            ],
            "default": "PT_CHLD_1-14_PS-PSY-V_CGVR",
        },
        "AREA_1": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD",
            ),
            "compare": "Sex",
            "indicators": [
                "PT_F_GE15_PS-SX-EM_V_PTNR_12MNTH",
                "PT_F_GE15_SX_V_PTNR_12MNTH",
                "PT_VC_VOV_PHYL",
                "PT_VC_VOV_SEXL",
                "PT_VC_VOV_ROBB",
                "PT_VC_SNS_WALN",
                "PT_CHLD_1-14_PS-PSY-V_CGVR",
                "PT_F_18-29_SX-V_AGE-18",
                "PT_M_18-29_SX-V_AGE-18",
                "PT_VC_PRR_PHYV",
                "PT_VC_PRR_SEXV",
                "PT_VC_PRR_ROBB",
                "PT_ADLT_PS_NEC",
                "PT_F_15-49_W-BTNG",
                "PT_M_15-49_W-BTNG",
                "PT_ST_13-15_BUL_30-DYS",
                "PT_F_15-19_MRD",
                "PT_M_15-19_MRD",
                "PT_F_20-24_MRD_U15",
                "PT_F_20-24_MRD_U18",
                "PT_M_20-24_MRD_U18",
                "PT_CHLD_5-17_LBR_ECON",
                "PT_CHLD_5-17_LBR_ECON-HC",
            ],
            "default": "PT_CHLD_1-14_PS-PSY-V_CGVR",
        },
        "AREA_2": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "indicators": [
                "PT_F_GE15_PS-SX-EM_V_PTNR_12MNTH",
                "PT_F_GE15_SX_V_PTNR_12MNTH",
                "PT_VC_VOV_PHYL",
                "PT_VC_VOV_SEXL",
                "PT_VC_VOV_ROBB",
                "PT_VC_SNS_WALN",
                "PT_CHLD_1-14_PS-PSY-V_CGVR",
                "PT_F_18-29_SX-V_AGE-18",
                "PT_M_18-29_SX-V_AGE-18",
                "PT_VC_PRR_PHYV",
                "PT_VC_PRR_SEXV",
                "PT_VC_PRR_ROBB",
                "PT_ADLT_PS_NEC",
                "PT_F_15-49_W-BTNG",
                "PT_M_15-49_W-BTNG",
                "PT_ST_13-15_BUL_30-DYS",
                "PT_F_15-19_MRD",
                "PT_M_15-19_MRD",
                "PT_F_20-24_MRD_U15",
                "PT_F_20-24_MRD_U18",
                "PT_M_20-24_MRD_U18",
                "PT_CHLD_5-17_LBR_ECON",
                "PT_CHLD_5-17_LBR_ECON-HC",
            ],
            "default_graph": "line",
            "default": "PT_CHLD_1-14_PS-PSY-V_CGVR",
        },
    },
    "CARE": {
        "NAME": "Children without parental care",
        "CARDS": [
            {
                "name": "in Residential Care",
                "indicator": "PT_CHLD_INRESIDENTIAL",
                "suffix": "Children",
            },
            # revise denominator: population children 0-17
            {
                "name": "in Residential Care",
                "indicator": "PT_CHLD_INRESIDENTIAL_RATE_B",
                "denominator": "EDUNF_SAP_L1T3",
                "suffix": "Percent of Children",
            },
            {
                "name": "in care of foster parents or guardians",
                "indicator": "PT_CHLD_INCARE_FOSTER",
                "suffix": "Children",
            },
            {
                "name": "available for adoption",
                "indicator": "PT_CHLD_ADOPTION_AVAILABLE",
                "suffix": "Children",
            },
        ],
        "MAIN": {
            "name": "Children without parental care",
            "geo": "Geographic area",
            "options": dict(
                lat="latitude",
                lon="longitude",
                size="OBS_VALUE",
                text="Geographic area",
                color="OBS_VALUE",
                color_continuous_scale=px.colors.sequential.GnBu,
                size_max=40,
                zoom=2.5,
                animation_frame="TIME_PERIOD",
                height=750,
            ),
            "indicators": [
                "PT_CHLD_INRESIDENTIAL",
                "PT_CHLD_INRESIDENTIAL_RATE_B",
                "PT_CHLD_DISAB_PUBLIC",
                "PT_CHLD_LEFTRESCARE",
                "PT_CHLD_LEFTRESCARE_RETURNED",
                "PT_CHLD_LEFTRESCARE_INFAMILY",
                "PT_CHLD_LEFTRESCARE_ADOPTED",
                "PT_CHLD_LEFTRESCARE_INDEPENDENT",
                "PT_CHLD_LEFTRESCARE_TRANSFERED",
                "PT_CHLD_LEFTRESCARE_DIED",
                "PT_CHLD_LEFTRESCARE_OTHER",
                "PT_CHLD_CARED_BY_FOSTER",
                "PT_CHLD_CARED_BY_FOSTER_RATE",
                "PT_CHLD_DISAB_FOSTER",
                "PT_CHLD_CARED_GUARDIAN",
                "PT_CHLD_CARED_GUARDIAN_RATE",
                "PT_CHLD_DISAB_CARED_GUARDIAN",
                "PT_CHLD_GUARDIAN",
                "PT_CHLD_ENTEREDFOSTER",
                "PT_CHLD_ADOPTION_RATE",
                "PT_CHLD_ADOPTION",
                "PT_CHLD_ADOPTION_DISAB",
                "PT_CHLD_ADOPTION_AVAILABLE",
                "PT_CHLD_ADOPTION_AVAILABLE_DISAB",
            ],
        },
        "AREA_1": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD",
            ),
            # compare is the default selection
            "compare": "Sex",
            "default": "PT_CHLD_INRESIDENTIAL",
            "indicators": [
                "PT_CHLD_INRESIDENTIAL",
                "PT_CHLD_INRESIDENTIAL_RATE_B",
                "PT_CHLD_DISAB_PUBLIC",
                "PT_CHLD_LEFTRESCARE",
                "PT_CHLD_LEFTRESCARE_RETURNED",
                "PT_CHLD_LEFTRESCARE_INFAMILY",
                "PT_CHLD_LEFTRESCARE_ADOPTED",
                "PT_CHLD_LEFTRESCARE_INDEPENDENT",
                "PT_CHLD_LEFTRESCARE_TRANSFERED",
                "PT_CHLD_LEFTRESCARE_DIED",
                "PT_CHLD_LEFTRESCARE_OTHER",
                "PT_CHLD_CARED_BY_FOSTER",
                "PT_CHLD_CARED_BY_FOSTER_RATE",
                "PT_CHLD_DISAB_FOSTER",
                "PT_CHLD_CARED_GUARDIAN",
                "PT_CHLD_CARED_GUARDIAN_RATE",
                "PT_CHLD_DISAB_CARED_GUARDIAN",
                "PT_CHLD_GUARDIAN",
                "PT_CHLD_ENTEREDFOSTER",
                "PT_CHLD_ADOPTION_RATE",
                "PT_CHLD_ADOPTION",
                "PT_CHLD_ADOPTION_DISAB",
                "PT_CHLD_ADOPTION_AVAILABLE",
                "PT_CHLD_ADOPTION_AVAILABLE_DISAB",
            ],
        },
        "AREA_2": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "indicators": [
                "PT_CHLD_INRESIDENTIAL",
                "PT_CHLD_INRESIDENTIAL_RATE_B",
                "PT_CHLD_DISAB_PUBLIC",
                "PT_CHLD_LEFTRESCARE",
                "PT_CHLD_LEFTRESCARE_RETURNED",
                "PT_CHLD_LEFTRESCARE_INFAMILY",
                "PT_CHLD_LEFTRESCARE_ADOPTED",
                "PT_CHLD_LEFTRESCARE_INDEPENDENT",
                "PT_CHLD_LEFTRESCARE_TRANSFERED",
                "PT_CHLD_LEFTRESCARE_DIED",
                "PT_CHLD_LEFTRESCARE_OTHER",
                "PT_CHLD_CARED_BY_FOSTER",
                "PT_CHLD_CARED_BY_FOSTER_RATE",
                "PT_CHLD_DISAB_FOSTER",
                "PT_CHLD_CARED_GUARDIAN",
                "PT_CHLD_CARED_GUARDIAN_RATE",
                "PT_CHLD_DISAB_CARED_GUARDIAN",
                "PT_CHLD_GUARDIAN",
                "PT_CHLD_ENTEREDFOSTER",
                "PT_CHLD_ADOPTION_RATE",
                "PT_CHLD_ADOPTION",
                "PT_CHLD_ADOPTION_DISAB",
                "PT_CHLD_ADOPTION_AVAILABLE",
                "PT_CHLD_ADOPTION_AVAILABLE_DISAB",
            ],
            "default_graph": "line",
            "default": "PT_CHLD_INRESIDENTIAL",
        },
    },
    "JUSTICE": {
        "NAME": "Access to Justice",
        "CARDS": [
            {
                "name": "committed against children during the year",
                "indicator": "JJ_CHLD_CRIME",
                "suffix": "Registered crimes",
            },
            {
                "name": "who are reported as being in contact with the police because of their own behaviour during the year",
                "indicator": "JJ_CHLD_POLICE",
                "suffix": "Children",
            },
            {
                "name": "who are charged with an offence or crime during the year",
                "indicator": "JJ_CHLD_OFFENCE",
                "suffix": "Children",
            },
            # revise denominator: population children 0-17
            # we don't have the ability yet to deal with rates that are not percentage
            # {
            #     "name": "committed against children (per 100,000 population aged 0-17)",
            #     "indicator": "JJ_CHLD_CRIMERT",
            #     "denominator": "EDUNF_SAP_L1T3",
            #     "suffix": "Registered crimes",
            # },
            # revise denominator: population children 14-17
            # we don't have the ability yet to deal with rates that are not percentage
            # {
            #     "name": "who are sentenced (per 100,000 population aged 14-17)",
            #     "indicator": "JJ_CHLD_SENTENCERT",
            #     "denominator": "EDUNF_SAP_L3",
            #     "suffix": "Children",
            # },
        ],
        "MAIN": {
            "name": "Child Victims of Crime",
            "geo": "Geographic area",
            "options": dict(
                lat="latitude",
                lon="longitude",
                size="OBS_VALUE",
                text="Geographic area",
                color="OBS_VALUE",
                color_continuous_scale=px.colors.sequential.GnBu,
                size_max=40,
                zoom=2.5,
                animation_frame="TIME_PERIOD",
                height=750,
            ),
            "indicators": [
                "JJ_CHLD_CRIME",
                "JJ_CHLD_CRIMERT",
                "JJ_CHLD_DETENTION",
                "JJ_CHLD_CONVICTED",
                "JJ_CHLD_SENTENCERT",
                "JJ_VC_PRS_UNSNT",
            ],
            "default": "JJ_CHLD_CRIME",
        },
        "AREA_1": {
            "type": "bar",
            "options": dict(
                x="Geographic area", y="OBS_VALUE", barmode="group", text="TIME_PERIOD",
            ),
            "compare": "Sex",
            "indicators": [
                "JJ_CHLD_CRIME",
                "JJ_CHLD_CRIMERT",
                "JJ_CHLD_DETENTION",
                "JJ_CHLD_CONVICTED",
                "JJ_CHLD_SENTENCERT",
                "JJ_VC_PRS_UNSNT",
            ],
            "default": "JJ_CHLD_CRIME",
        },
        "AREA_2": {
            "graphs": {
                "bar": {
                    "options": dict(
                        x="Geographic area",
                        y="OBS_VALUE",
                        barmode="group",
                        text="TIME_PERIOD",
                    ),
                    "compare": "Sex",
                },
                "line": {
                    "options": dict(
                        x="TIME_PERIOD",
                        y="OBS_VALUE",
                        color="Geographic area",
                        hover_name="Geographic area",
                        line_shape="spline",
                        render_mode="svg",
                    ),
                    "trace_options": dict(mode="lines+markers"),
                },
            },
            "indicators": [
                "JJ_CHLD_CRIME",
                "JJ_CHLD_CRIMERT",
                "JJ_CHLD_DETENTION",
                "JJ_CHLD_CONVICTED",
                "JJ_CHLD_SENTENCERT",
                "JJ_VC_PRS_UNSNT",
            ],
            "default_graph": "line",
            "default": "JJ_CHLD_CRIME",
        },
    },
}

### Indicators within all areas (both pages)

In [5]:
e_theme = ['PARTICIPATION', 'QUALITY', 'GOVERNANCE']
c_theme = ['VIOLENCE', 'CARE', 'JUSTICE']
keys = ['MAIN', 'AREA_1', 'AREA_2', 'AREA_3', 'AREA_4']

edu_ind = []
cp_ind = []
for i, topic in enumerate(e_theme):
    c_topic = c_theme[i]
    for key in keys:
        edu_ind.append(edu_dict.get(topic, {}).get(key, {}).get('indicators', {}))
        cp_ind.append(cp_dict.get(c_topic, {}).get(key, {}).get('indicators', {}))

#### Manually entered indicators from cards - structure is not a dict in config

In [6]:
edu_cards = [
    'EDUNF_OFST_L1',
    'EDUNF_OFST_L2',
    'EDUNF_OFST_L3',
    'EDUNF_OFST_L1',
    'EDUNF_OFST_L2',
    'EDUNF_OFST_L3',
    'EDUNF_OFST_L1_UNDER1',
    'EDUNF_RPTR_L1',
    'EDUNF_RPTR_L2',
    'EDUNF_ESL_L1',
    'EDUNF_ADMIN_L1_GLAST_REA',
    'EDUNF_ADMIN_L1_GLAST_MAT',
    'EDU_PISA_MAT',
    'EDU_PISA_REA',
    'EDU_PISA_SCI',
    'EDU_SDG_FREE_EDU_L02',
    'EDUNF_STU_L1_PRV',
    'EDUNF_STU_L2_PRV',
    'EDUNF_STU_L3_PRV',
    'EDUNF_TEACH_L1',
    'EDUNF_TEACH_L2',
    'EDUNF_TEACH_L3',
]
cp_cards = [
    'PT_CHLD_1-14_PS-PSY-V_CGVR',
    'EDUNF_SAP_L1T3',
    'PT_CHLD_INRESIDENTIAL',
    'PT_CHLD_INRESIDENTIAL_RATE_B',
    'EDUNF_SAP_L1T3',
    'PT_CHLD_INCARE_FOSTER',
    'PT_CHLD_ADOPTION_AVAILABLE',
    'JJ_CHLD_CRIME',
    'JJ_CHLD_POLICE',
    'JJ_CHLD_OFFENCE',
]

### Unique set of indicators for dashboards

In [7]:
edu_ind.append(edu_cards)
cp_ind.append(cp_cards)
unique_edu = set([item for sublist in edu_ind for item in sublist])
unique_cp = set([item for sublist in cp_ind for item in sublist])

In [8]:
# indicators called in init not used
print(np.setdiff1d(codes, list(set.union(unique_edu, unique_cp))))
# indicators not in init called in dash
print(np.setdiff1d(list(set.union(unique_edu, unique_cp)), codes, ))

[]
[]


In [ ]:
# Import required libraries
import pickle
import copy
import pathlib
import dash
import math
import datetime as dt
import pandas as pd
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go



In [ ]:
from mapbox import Geocoder
mapbox_access_token = "pk.eyJ1IjoiamNyYW53ZWxsd2FyZCIsImEiOiJja2NkMW02aXcwYTl5MnFwbjdtdDB0M3oyIn0.zkIzPc4NSjLZvrY-DWrlZg"

geocoder = Geocoder(access_token=mapbox_access_token)

def geocode_address(address):
    """Geocode street address into lat/long."""
    response = geocoder.forward(address)
    coords = response.json()["features"][0]["center"]
    longitude = coords[0]
    latitude = coords[1]
    return dict(longitude=longitude, latitude=latitude)

In [ ]:
sdmx_url = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.{}....?format=csv'

In [ ]:
df = pd.DataFrame()
inds = set(codes)
for ind in inds:
    sdmx = pd.read_csv(sdmx_url.format(ind))
    sdmx["CODE"] = ind
    df = df.append(sdmx)

In [ ]:
countries = df["Geographic area"].unique()

In [ ]:
df = df.merge(
    right=pd.DataFrame(
        [dict(country=country, **geocode_address(country)) for country in countries]
    ),
    left_on='Geographic area',
    right_on='country'
)

In [ ]:
df.columns

In [ ]:
data.merge(
    right=countries,
    left_on='Geographic area',
    right_on='name'
)

In [ ]:
# Create controls
county_options = [
    {"label": str(country), "value": str(country)} for country in data['Geographic area'].unique()
]

In [ ]:
county_options

In [ ]:
years =[i for i in range(2010, 2020)]

indicators = df['Indicator'].unique()

{index: str(year) for index,year in enumerate(years)}

In [ ]:
countries = df["Geographic area"].unique()


In [ ]:
numerator = "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3"
denominator = "EDUNF_SAP_L1T3"
absolute=True

In [ ]:
query = (
    "CODE in @indicator & TIME_PERIOD in @years & `Geographic area` in @countries"
)
numors = numerator.split(",")
indicator = numors
# select last value for each country
indicator_values = (
    df.query(query)
    .groupby(
        [
            "CODE",
            "Indicator",
            "Geographic area",
            "UNIT_MEASURE",
        ]
    )
    .agg({"TIME_PERIOD": "last", "OBS_VALUE": "last"})
    .reset_index()
    .set_index(["Geographic area", "TIME_PERIOD"])
)
# select the avalible denominators for countiries in selected years
indicator = [denominator]
denominator_values = (
    df.query(query)
    .groupby(
        [
            "CODE",
            "Indicator",
            "Geographic area",
            "UNIT_MEASURE",
        ]
    )
    .agg({"TIME_PERIOD": "last", "OBS_VALUE": "last"})
    .reset_index()
    .set_index(["Geographic area", "TIME_PERIOD"])
)
# select only those denominators that match avalible indicators
denominators = denominator_values[
    denominator_values.index.isin(indicator_values.index)
]["OBS_VALUE"]

denominator_sum = denominators.to_numpy().sum()

indicator_sum = (
    indicator_values["OBS_VALUE"] * denominator_sum
    if absolute
    else (denominators / denominator_sum)
).dropna()  # will drop missing countires

sources = indicator_sum.index.tolist()


In [ ]:
indicator_values

In [ ]:
denominator_values

In [ ]:
indicator_sum

In [ ]:
sources

In [ ]:
indicator_sum.to_numpy().sum()

In [ ]:
indicator_values.head()

In [ ]:
# select the avalible denominators for countiries in selected years
indicator = [denominator]
denominator_values = (
    df.query(query)
    .groupby(["CODE", "Indicator", "Geographic area", "UNIT_MEASURE", "Unit of measure"])
    .agg({"TIME_PERIOD": "last", "OBS_VALUE": "last"})
    .reset_index()
    .set_index(["Geographic area", "TIME_PERIOD"])
)
denominator_values

In [ ]:
# select only those denominators that match avalible indicators
denominators = denominator_values[
    denominator_values.index.isin(indicator_values.index)
]["OBS_VALUE"]
denominators.index.tolist()


In [ ]:
(indicator_values["OBS_VALUE"] * (denominators / denominator_sum))

In [ ]:

indicator_sum = (indicator_values["OBS_VALUE"] * (denominators / denominators.to_numpy().sum())).dropna().to_numpy().sum()


In [ ]:
indicator_sum